In [1]:
#Import the necessary libraries
import pandas as pd
import numpy as np
import scipy as sp
import re
import nltk
import gc
from lda import guidedlda as glda
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from copy import deepcopy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import pairwise_distances

In [2]:
alldata = pd.read_csv('steam_reviews_kaggle.csv')
SentimentData=['review','recommendation']
data=alldata[SentimentData]
data=data.dropna(subset=['review'])
del alldata
gc.collect()

7

In [4]:
def preprocess_review(review):
    #Remove HTML Tags
    review_process = BeautifulSoup(review,'lxml').get_text()
    
    #Remove non-letter characters
    review_process = re.sub('[^a-zA-Z]', ' ', review_process)
    
    #Convert to lower case
    review_process = review_process.lower()
    
    #Remove stopwords
    words = review_process.split()
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if w not in stops]
    clean_review = " ".join(meaningful_words)

    return clean_review

In [5]:
#Clean the reviews and add them to the list below
cleaned_reviews = []

for raw in data['review']:
    clean = preprocess_review(raw)
    cleaned_reviews.append(clean)

C:\Users\alanl\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\alanl\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\alanl\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\alanl\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'.....'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\alanl\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarni

C:\Users\alanl\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'........................'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\alanl\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'. '" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [20]:
del model
del X_counts
gc.collect()

113

In [21]:
def design_matrix(cleaned_reviews):
    vectorizer = CountVectorizer(analyzer = "word", 
                                 tokenizer = None, 
                                 preprocessor = None, 
                                 stop_words = None,
                                 max_features = 7000)
    X_counts = vectorizer.fit_transform(cleaned_reviews).toarray()

    return X_counts,vectorizer

In [22]:
X_counts,vectorizer = design_matrix(cleaned_reviews)

In [23]:
word2id = vectorizer.vocabulary_

### Guided LDA

In [24]:
# seed topic word
seed_topic_list = [['gameplay','mechanics', 'combat', 'fps', 'survive', 'shooting', 'online', 'single', 'multiplayer'],
                   ['money', 'free', 'price', 'pay', 'dlc', 'skins'],
                   ['server', 'fix', 'bugs', 'lag', 'potato','connection'],
                   ['cheat', 'hackers', 'aimbot'],
                   ['cpu', 'gpu', 'laptop', 'ram', 'hardware', 'crash'],
                   ['friends', 'teammates'],
                   ['story', 'experience', 'sound', 'physics', 'music'],
                   ['naked', 'nudity', 'blood', 'racist', 'idiots', 'noobs'],
                   ['happy', 'recommend', 'favorite', 'great', 'nice', 'amazing', 'awesome', 'perfect', 'simple', 'fantastic'],
                   ['sick','tired', 'disappointed','worst', 'trash', 'stupid', 'hell', 'garbage'],
                   ['alpha','early', 'new', 'future', 'patch']]

seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        if word in word2id.keys():
            seed_topics[word2id[word]] = t_id

In [25]:
seed_topics

{2545: 0,
 3791: 0,
 1121: 0,
 2442: 0,
 6104: 0,
 5557: 0,
 4234: 0,
 5630: 0,
 4001: 0,
 3949: 1,
 2461: 1,
 4703: 1,
 4412: 1,
 1738: 1,
 5654: 1,
 5489: 2,
 2328: 2,
 793: 2,
 3407: 2,
 4633: 2,
 1227: 2,
 982: 3,
 2777: 3,
 152: 3,
 1348: 4,
 2674: 4,
 3431: 4,
 4905: 4,
 2820: 4,
 1358: 4,
 2479: 5,
 6195: 5,
 5963: 6,
 2128: 6,
 5780: 6,
 4484: 6,
 4010: 6,
 4023: 7,
 4157: 7,
 646: 7,
 4880: 7,
 3050: 7,
 4125: 7,
 2812: 8,
 4998: 8,
 2239: 8,
 2697: 8,
 4096: 8,
 209: 8,
 416: 8,
 4445: 8,
 5614: 8,
 2218: 8,
 5594: 9,
 6340: 9,
 1692: 9,
 6920: 9,
 6423: 9,
 6019: 9,
 2885: 9,
 2559: 9,
 194: 10,
 1872: 10,
 4081: 10,
 2521: 10,
 4400: 10}

In [15]:
del data
del model
gc.collect()

34

In [26]:
# fit a guided LDA
model = glda.GuidedLDA(n_topics=15, n_iter=100, random_state=0, refresh=20)
model.fit(X_counts, seed_topics=seed_topics, seed_confidence=0.15)

INFO:lda:n_documents: 433375
INFO:lda:vocab_size: 7000
INFO:lda:n_words: 8566214
INFO:lda:n_topics: 15
INFO:lda:n_iter: 100
INFO:lda:<0> log likelihood: -96644145
INFO:lda:<20> log likelihood: -66586691
INFO:lda:<40> log likelihood: -64838946
INFO:lda:<60> log likelihood: -64295892
INFO:lda:<80> log likelihood: -64026960
INFO:lda:<99> log likelihood: -63837612


In [27]:
topic_word = model.topic_word_

In [29]:
topic_word.shape

(15, 7000)

In [30]:
id2word = {}
for w, n in word2id.items():
    id2word[n] = w

In [59]:
doc_topic = model.transform(X_counts)

In [73]:
doc_topic.shape

(433375, 15)

### Create features only considering special words

In [165]:
wordsort = np.argsort(topic_word,axis=1) # sorted by probability of word in each cluster

In [166]:
wordsort_t500 = wordsort[:,6500:] # select the top 500 word in each cluster

In [167]:
unique_id,id_count=np.unique(wordsort_t500,return_counts=True) # return the word id and total number of times it appears

In [176]:
# pan word: word that appears in diffrent topics' top 500
# special word: word that only appears at most 2 topics' top 500

special_word = []
for w,n in dict(zip(unique_id,id_count)).items():
    if n<=2:
        special_word.append(w)

In [177]:
for i in range(15):
    top15_word = []
    present=0
    j=-1
    while present<15:
        wordid = list(wordsort_t500[i])[j]
        j = j-1
        if wordid in special_word:
            top15_word.append(id2word[wordid])
            present = present+1
    print(top15_word)

['modding', 'singleplayer', 'openiv', 'modders', 'grand', 'theft', 'shark', 'heists', 'cards', 'iv', 'heist', 'campaign', 'city', 'kappa', 'san']
['modding', 'openiv', 'shark', 'cards', 'singleplayer', 'modders', 'de', 'https', 'que', 'www', 'micro', 'machine', 'greene', 'brendan', 'gamefailed']
['average', 'network', 'mw', 'potato', 'gtx', 'gb', 'medium', 'minimum', 'busy', 'masterpiece', 'nasa', 'frame', 'optimize', 'esports', 'dark']
['region', 'cheater', 'hacks', 'lock', 'hack', 'english', 'cheats', 'report', 'asian', 'aimbot', 'streamers', 'stream', 'speak', 'fu', 'ruining']
['gb', 'gtx', 'ram', 'port', 'mouse', 'specs', 'cpu', 'laptop', 'ti', 'gpu', 'windows', 'amd', 'ultra', 'nvidia', 'lowest']
['platform', 'riot', 'fifa', 'lovers', 'famous', 'awsome', 'streaming', 'streamer', 'justice', 'epic', 'entertaining', 'teenagers', 'kai', 'protest', 'repeated']
['gt', 'monster', 'hunter', 'monsters', 'mh', 'capcom', 'mouse', 'hunt', 'port', 'quest', 'previous', 'hunting', 'quests', 'des

In [178]:
len(special_word)

937

In [184]:
special_word_location =[]  #store (cluster, word_id), which represent the location in topic_word

for wordid in special_word:
    for clusterid in np.argwhere(wordsort_t500==wordid)[:,0]:
        special_word_location.append((clusterid,wordid))

In [197]:
new_topic_word1 = np.zeros(topic_word.shape)  # store probability
new_topic_word2 = np.zeros(topic_word.shape)  # store 0 or 1

In [198]:
for location in special_word_location:
    new_topic_word1[location[0], location[1]] = topic_word[location[0], location[1]]
    new_topic_word2[location[0], location[1]] = 1

In [200]:
new_topic_word1 = new_topic_word1/np.sum(new_topic_word1,axis=1).reshape((15,1))

In [202]:
doc_topic_enhance = (X_counts.dot(new_topic_word1.T)) / (X_counts.dot(new_topic_word2.T))

C:\Users\alanl\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [208]:
doc_topic_enhance[np.isnan(doc_topic_enhance)] = 0 

In [211]:
alldata = pd.read_csv('steam_reviews.csv')
SentimentData=['review','recommendation']
data=alldata[SentimentData]
data=data.dropna(subset=['review'])
del alldata
gc.collect()
data.recommendation[data.recommendation=='Recommended']=1
data.recommendation[data.recommendation=='Not Recommended']=0

In [215]:
lda_set = data[['recommendation']].copy()

In [216]:
for i in range(15):
    lda_set['t'+str(i)+'_enhance'] = doc_topic_enhance[:,i]
    lda_set['t'+str(i)] =doc_topic[:,i]

In [218]:
# save the dataset with LDA features and labels
lda_set.to_csv('lda_set.csv',index=False)

In [212]:
# save the fitted LDA models

from six.moves import cPickle as pickle

with open('guidedlda_model.pickle', 'wb') as file_handle:
    pickle.dump(model, file_handle)